# Loading a DSE Vector




## Configuring CITE libraries for almond kernel

First, we'll make a bintray repository with CITE libraries available to your almond kernel.

In [97]:
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)

myBT: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

Next, we bring in specific libraries from the new repository using almond's `$ivy` magic:

In [98]:
import $ivy.`edu.holycross.shot::ohco2:10.16.0`
import $ivy.`edu.holycross.shot.cite::xcite:4.1.1`
import $ivy.`edu.holycross.shot::scm:7.2.0`
import $ivy.`edu.holycross.shot::dse:5.2.2`
import $ivy.`edu.holycross.shot::citebinaryimage:3.1.1`
import $ivy.`edu.holycross.shot::citeobj:7.3.4`
import $ivy.`edu.holycross.shot::citerelations:2.5.2`
import $ivy.`edu.holycross.shot::cex:6.3.3`


import $ivy.$                                  

import $ivy.$                                     

import $ivy.$                              

import $ivy.$                              

import $ivy.$                                          

import $ivy.$                                  

import $ivy.$                                        

import $ivy.$                              


## Imports

From this point on, your notebook consists of completely generic Scala, with the CITE Libraries available to use.

In [99]:
// Import some CITE libraries
import edu.holycross.shot.cite._
import edu.holycross.shot.ohco2._
import edu.holycross.shot.scm._
import edu.holycross.shot.citeobj._
import edu.holycross.shot.citerelation._
import edu.holycross.shot.dse._
import edu.holycross.shot.citebinaryimage._
import edu.holycross.shot.ohco2._

import almond.display.UpdatableDisplay
import almond.interpreter.api.DisplayData.ContentType
import almond.interpreter.api.{DisplayData, OutputHandler}

import edu.holycross.shot.cite._

import edu.holycross.shot.ohco2._

import edu.holycross.shot.scm._

import edu.holycross.shot.citeobj._

import edu.holycross.shot.citerelation._

import edu.holycross.shot.dse._

import edu.holycross.shot.citebinaryimage._

import edu.holycross.shot.ohco2._


import almond.display.UpdatableDisplay

import almond.interpreter.api.DisplayData.ContentType

import almond.interpreter.api.{DisplayData, OutputHandler}

## Load a Template File

In [100]:
val templateFiles: Vector[String] = Vector(
"new_dse_template_186r.txt",
"new_dse_template_83r.txt",
"new_dse_template_181r.txt",
"new_dse_template_182r.txt",
"new_dse_template_183r.txt",
"new_dse_template_184r.txt",
"new_dse_template_185r.txt",
"new_dse_template_187r.txt",
"new_dse_template_188r.txt",
"new_dse_template_190r.txt"
)



templateFiles: Vector[String] = Vector(
  "new_dse_template_186r.txt",
  "new_dse_template_83r.txt",
  "new_dse_template_181r.txt",
  "new_dse_template_182r.txt",
  "new_dse_template_183r.txt",
  "new_dse_template_184r.txt",
  "new_dse_template_185r.txt",
  "new_dse_template_187r.txt",
  "new_dse_template_188r.txt",
  "new_dse_template_190r.txt"
)

Pick one template from the list:

In [101]:
val thisTemplate: String = templateFiles(8)

thisTemplate: String = "new_dse_template_188r.txt"

Load it:

In [102]:
val filePath = s"https://raw.githubusercontent.com/hmteditors/dse_corrections_thumbprint_images/master/templates/${thisTemplate}"
val lines: Vector[String] = scala.io.Source.fromURL(filePath).mkString.split("\n").toVector

filePath: String = "https://raw.githubusercontent.com/hmteditors/dse_corrections_thumbprint_images/master/templates/new_dse_template_188r.txt"
lines: Vector[String] = Vector(
  "urn:cite2:hmt:va_dse.v1:schol5262#DSE record for scholion msA 14.D1#urn:cts:greekLit:tlg5026.msA.dipl:14.D1#urn:cite2:hmt:vaimg.2017a:VA188RN_0359@0.5822,0.2422,0.1971,0.06058#urn:cite2:hmt:msA.v1:188r",
  "urn:cite2:hmt:va_dse.v1:schol5263#DSE record for scholion msA 14.D2#urn:cts:greekLit:tlg5026.msA.dipl:14.D2#urn:cite2:hmt:vaimg.2017a:VA188RN_0359@0.5759,0.3008,0.2067,0.2595#urn:cite2:hmt:msA.v1:188r",
  "urn:cite2:hmt:va_dse.v1:schol5264#DSE record for scholion msA 14.D3#urn:cts:greekLit:tlg5026.msA.dipl:14.D3#urn:cite2:hmt:vaimg.2017a:VA188RN_0359@NEWROI#urn:cite2:hmt:msA.v1:188r",
  "urn:cite2:hmt:va_dse.v1:schol5265#DSE record for scholion msA 14.D4#urn:cts:greekLit:tlg5026.msA.dipl:14.D4#urn:cite2:hmt:vaimg.2017a:VA188RN_0359@NEWROI#urn:cite2:hmt:msA.v1:188r",
  "urn:cite2:hmt:va_dse.v1:schol5266#DSE r

Define some Classes

In [103]:
case class DseTriple( passage: CtsUrn, imageroi: Cite2Urn, surface: Cite2Urn)
case class DseRecord( urn: Cite2Urn, label: String, triple: DseTriple) {
	override def toString = {
		s"${this.urn}#${this.label}#${this.triple.passage}#${this.triple.imageroi}#${this.triple.surface}"
	}
}


defined class DseTriple
defined class DseRecord

## Initial Validation

If the following step fails, you have bad data:

In [77]:
val records: Vector[DseRecord] = {
    lines.map( l => {
       val urn = Cite2Urn(l.split("#").toVector(0))
       val label: String = l.split("#").toVector(1)
       val p = CtsUrn(l.split("#").toVector(2))	
       val i = Cite2Urn(l.split("#").toVector(3))
       val s = Cite2Urn(l.split("#").toVector(4))
       DseRecord(urn, label, DseTriple(p, i, s))
    })
}

records: Vector[DseRecord] = Vector(
  DseRecord(
    Cite2Urn("urn:cite2:hmt:va_dse.v1:schol5262"),
    "DSE record for scholion msA 14.D1",
    DseTriple(
      CtsUrn("urn:cts:greekLit:tlg5026.msA.dipl:14.D1"),
      Cite2Urn(
        "urn:cite2:hmt:vaimg.2017a:VA188RN_0359@0.5822,0.2422,0.1971,0.06058"
      ),
      Cite2Urn("urn:cite2:hmt:msA.v1:188r")
    )
  ),
  DseRecord(
    Cite2Urn("urn:cite2:hmt:va_dse.v1:schol5263"),
    "DSE record for scholion msA 14.D2",
    DseTriple(
      CtsUrn("urn:cts:greekLit:tlg5026.msA.dipl:14.D2"),
      Cite2Urn(
        "urn:cite2:hmt:vaimg.2017a:VA188RN_0359@0.5759,0.3008,0.2067,0.2595"
      ),
      Cite2Urn("urn:cite2:hmt:msA.v1:188r")
    )
  ),
  DseRecord(
    Cite2Urn("urn:cite2:hmt:va_dse.v1:schol5264"),
    "DSE record for scholion msA 14.D3",
    DseTriple(
      CtsUrn("urn:cts:greekLit:tlg5026.msA.dipl:14.D3"),
      Cite2Urn("urn:cite2:hmt:vaimg.2017a:VA188RN_0359@NEWROI"),
      Cite2Urn("urn:cite2:hmt:msA.v1:188r")
    )
  

Initial report:

In [90]:
val validRecords: Vector[DseRecord] = records.filter( r => {
    try {
        val i: Cite2Urn = r.triple.imageroi
        val ext: String = i.objectExtension
        val roi = ImageROI(ext)
        true // if we get here, keep it!
    } catch {
        case e: Exception => println(s"Bad Image URN for Object ${r.urn}:\n\t${r.triple.imageroi}\n\t${e}")
        false
    }
})

Bad Image URN for Object urn:cite2:hmt:va_dse.v1:schol5264:
	urn:cite2:hmt:vaimg.2017a:VA188RN_0359@NEWROI
	java.lang.ArrayIndexOutOfBoundsException: Index 1 out of bounds for length 1
Bad Image URN for Object urn:cite2:hmt:va_dse.v1:schol5265:
	urn:cite2:hmt:vaimg.2017a:VA188RN_0359@NEWROI
	java.lang.ArrayIndexOutOfBoundsException: Index 1 out of bounds for length 1
Bad Image URN for Object urn:cite2:hmt:va_dse.v1:schol5266:
	urn:cite2:hmt:vaimg.2017a:VA188RN_0359@NEWROI
	java.lang.ArrayIndexOutOfBoundsException: Index 1 out of bounds for length 1
Bad Image URN for Object urn:cite2:hmt:va_dse.v1:schol5267:
	urn:cite2:hmt:vaimg.2017a:VA188RN_0359@NEWROI
	java.lang.ArrayIndexOutOfBoundsException: Index 1 out of bounds for length 1
Bad Image URN for Object urn:cite2:hmt:va_dse.v1:schol5268:
	urn:cite2:hmt:vaimg.2017a:VA188RN_0359@NEWROI
	java.lang.ArrayIndexOutOfBoundsException: Index 1 out of bounds for length 1
Bad Image URN for Object urn:cite2:hmt:va_dse.v1:schol5337:
	urn:cite2:hmt:

validRecords: Vector[DseRecord] = Vector(
  DseRecord(
    Cite2Urn("urn:cite2:hmt:va_dse.v1:schol5262"),
    "DSE record for scholion msA 14.D1",
    DseTriple(
      CtsUrn("urn:cts:greekLit:tlg5026.msA.dipl:14.D1"),
      Cite2Urn(
        "urn:cite2:hmt:vaimg.2017a:VA188RN_0359@0.5822,0.2422,0.1971,0.06058"
      ),
      Cite2Urn("urn:cite2:hmt:msA.v1:188r")
    )
  ),
  DseRecord(
    Cite2Urn("urn:cite2:hmt:va_dse.v1:schol5263"),
    "DSE record for scholion msA 14.D2",
    DseTriple(
      CtsUrn("urn:cts:greekLit:tlg5026.msA.dipl:14.D2"),
      Cite2Urn(
        "urn:cite2:hmt:vaimg.2017a:VA188RN_0359@0.5759,0.3008,0.2067,0.2595"
      ),
      Cite2Urn("urn:cite2:hmt:msA.v1:188r")
    )
  )
)

In [96]:
Html(s"""<p style="color: blue; font-size: 200%; display: inline-block; min-height: 2em; padding-top: 1em;">There are <strong>${validRecords.size}</strong> valid records, out of ${records.size} records.</p>""")

There are 2 valid records, out of 22 records.

## Verification

Generate a link to the ICT showing all current, valid ROIs.

In [89]:
val urlBase = "http://www.homermultitext.org/ict2/index.html?urn="
val urnParams = validRecords.map(_.triple.imageroi).mkString("&urn=")
Html(s"""<a style="font-size: 200%; min-height: 2em; display: inline-block; padding-top: 1em; padding-bottom: 1em;}" href="${urlBase}${urnParams}">Verification Link</a>""")

Verification Link

urlBase: String = "http://www.homermultitext.org/ict2/index.html?urn="
urnParams: String = "urn:cite2:hmt:vaimg.2017a:VA188RN_0359@0.5822,0.2422,0.1971,0.06058&urn=urn:cite2:hmt:vaimg.2017a:VA188RN_0359@0.5759,0.3008,0.2067,0.2595"